In [ ]:
import pandas as pd
import psycopg2 as ppg2

<!-- psql -U aabounak -d piscineds -h localhost -W -->

In [45]:
def connect_to_postgres():
  con = ppg2.connect(
    database="piscineds",
    user="aabounak",
    password="mysecretpassword",
    host="localhost",
    port="5432"
  )
  return con

## Module 00


In [46]:
connection = connect_to_postgres()
cursor_obj = connection.cursor()

In [47]:
customers_csvs = [
  "data_2022_oct.csv",
  "data_2022_nov.csv",
  "data_2022_dec.csv",
  "data_2023_jan.csv",
  "data_2023_feb.csv",
]

for file in customers_csvs:
  table_name = f"{file.split('.')[0]}"
  sql_query = f"""
    CREATE TABLE IF NOT EXISTS public."{table_name}" (
      event_time TIMESTAMP WITH TIME ZONE,
      event_type VARCHAR(50),
      product_id INTEGER,
      price NUMERIC(10, 2),
      user_id INTEGER,
      user_session VARCHAR(100)
    );
  """
  cursor_obj.execute(sql_query)
  connection.commit()

# cursor_obj.close()

In [48]:
# cursor_obj = connection.cursor()

customers_csvs = [
    "data_2022_dec.csv"
]
for file in customers_csvs:
  table_name = f"{file.split('.')[0]}"
  file_path = f"./customers/{file}"

  with open(file_path, 'r') as f:
    next(f)
    cursor_obj.copy_from(
      f,
      table_name,
      sep=',',
      null='',
      columns=(
        'event_time',
        'event_type',
        'product_id',
        'price',
        'user_id',
        'user_session'))
    connection.commit()
    
# cursor_obj.close()

### Exercise 04


In [35]:
# cursor_obj = connection.cursor()

table_name = "items"

sql_query = f"""
  CREATE TABLE IF NOT EXISTS public."{table_name}" (
    product_id INTEGER,
    category_id BIGINT,
    category_code TEXT,
    brand TEXT
  );
"""
cursor_obj.execute(sql_query)
connection.commit()

file_path = "./items/item.csv"

with open(file_path, 'r') as f:
  next(f)
  cursor_obj.copy_from(
    f,
    table_name,
    sep=',',
    null='',
    columns=(
      'product_id',
      'category_id',
      'category_code',
      'brand'
    )
  )
connection.commit()

# cursor_obj.close()

## Module 01


### Exercise 01

In [41]:
# cursor_obj = connection.cursor()

sql_query = f"""
  CREATE TABLE IF NOT EXISTS customers AS
  SELECT * FROM data_2022_dec
  UNION ALL
  SELECT * FROM data_2022_nov
  UNION ALL
  SELECT * FROM data_2022_oct
  UNION ALL
  SELECT * from data_2023_jan
  UNION ALL
  SELECT * from data_2023_feb;
"""

cursor_obj.execute(sql_query)
connection.commit()

# cursor_obj.close()

### Exercise 02

In [ ]:
sql_query = f"""
  DELETE FROM customers
  WHERE (event_time, event_type, product_id, price, user_id, user_session) IN (
    SELECT 
      event_time, 
      event_type, 
      product_id, 
      price,
      user_id,
      user_session
    FROM customers
    GROUP BY 
      event_time, 
      event_type, 
      product_id, 
      price,
      user_id,
      user_session
    HAVING COUNT(*) > 1
  );
"""

cursor_obj.execute(sql_query)
connection.commit()

### Exercise 03

#### This is not working due to PERFORMANCE ISSUES

In [49]:
sql_query = """
  CREATE TABLE combined_data AS
  SELECT * FROM data_2022_dec
  INNER JOIN items ON data_2022_dec.product_id = items.product_id;
"""

cursor_obj.execute(sql_query)
connection.commit()

<!-- CREATE TABLE combined_table AS
SELECT
    c.event_time,
    c.event_type,
    c.price,
    c.user_id,
    c.user_session,
    i.product_id,
    i.category_id,
    i.category_code,
    i.brand
FROM data_2022_dec c
JOIN items i ON c.product_id = i.product_id
GROUP BY
    c.event_time,
    c.event_type,
    c.price,
    c.user_id,
    c.user_session,
    i.product_id,
    i.category_id,
    i.category_code,
    i.brand; -->


In [ ]:
cursor_obj.close()

<!-- SELECT
    customers.event_time,
    customers.event_type,
    customers.price,
    customers.user_id,
    customers.user_session,
    items.product_id,
    items.category_id,
    items.category_code,
    items.brand
FROM customers
JOIN items ON customers.product_id = items.product_id
GROUP BY
    customers.event_time,
    customers.event_type,
    customers.price,
    customers.user_id,
    customers.user_session,
    items.product_id,
    items.category_id,
    items.category_code,
    items.brand; -->
